In [1]:
import importlib
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from flearn.utils.model_utils import read_data
import os
import importlib
import random
import numpy as np

from flearn.models import client
from flearn.models import group
from flearn.trainers import grouprox
from flearn.trainers import fedprox
from flearn.trainers import fedavg

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
params = {}
#DATASETS = ['sent140', 'nist', 'shakespeare', 'mnist', 'synthetic_iid', 'synthetic_0_0', 'synthetic_0.5_0.5', 'synthetic_1_1']

"""
Parameters that need to be manually set
"""

#params['dataset'] = 'synthetic_1_1'
params['dataset'] = 'sent140'
#params['model'] = 'mclr'
params['model'] = 'stacked_lstm'
params['optimizer'] = 'grouprox'
params['drop_percent'] = 0

if params['dataset'].startswith('mnist'):
  params['model_params'] = (10,)
  params['learning_rate'] = 0.03
  params['mu'] = 1
if params['dataset'] == 'nist':
  params['model_params'] = (26,)
  params['learning_rate'] = 0.003
  params['mu'] = 1
if params['dataset'] == 'sent140':
  params['model_params'] = (25, 2, 100) # seq_len, num_classes, num_hidden
  params['learning_rate'] = 0.3
  params['mu'] = 0.01
if params['dataset'].startswith('synthetic'):
  params['model_params'] = (10,)
  params['learning_rate'] = 0.01
  params['mu'] = 1

if params['optimizer']  == 'fedavg':
  pass
if params['optimizer']  == 'fedprox':
  pass
if params['optimizer']  == 'grouprox':
  params['proximal'] = True
  params['randomly'] = False
  params['allow_empty'] = True
  params['evenly'] = False
  params['num_group'] = 5
  params['min_clients'] = 2


"""
Common parameters
"""
params['num_rounds'] = 800
params['num_epochs'] = 20
params['batch_size'] = 10
params['eval_every'] = 1
params['clients_per_round'] = 20 ######################################################
params['seed'] = 233
params['agg_lr'] = 0.01


""" Set the output CSV file name """
info = '{}-{}-{}'.format(params['optimizer'], params['dataset'], params['model'])
info2 = ''
suffix = '.csv'
if params['optimizer'] == 'fedavg':
  pass
if params['optimizer'] == 'fedprox':
  pass
if params['optimizer'] == 'grouprox':
  info2 = '-NumG{:1d}-Agglr{:.4}-Proximal{:.1}'.format(
    params['num_group'], params['agg_lr'], str(params['proximal']))
  if params['randomly'] == True:
    info2 = info2 + '-Randomly{:.1}-Evenly{:.1}'.format(str(params['randomly']), str(params['evenly']))
  elif params['allow_empty'] == True:
    info2 = info2 + '-Empty{:.1}'.format(str(params['allow_empty']))
  elif params['allow_empty'] == False:
    info2 = info2 + '-Evenly{:.1}'.format(str(params['evenly']))

params['export_filename'] = info + info2 + suffix

print(params['export_filename'])

grouprox-sent140-stacked_lstm-NumG5-Agglr0.01-ProximalT-EmptyT.csv


In [3]:
"""
Some preparations
"""

# read data
train_path = os.path.join('data', params['dataset'], 'data', 'train')
test_path = os.path.join('data', params['dataset'], 'data', 'test')
dataset = read_data(train_path, test_path)

# Load model
if params['dataset'].startswith('synthetic'):  # all synthetic datasets use the same model
  model_path = '%s.%s.%s.%s' % ('flearn', 'models', 'synthetic', params['model'])
elif params['dataset'].startswith('mnist'):
  model_path = '%s.%s.%s.%s' % ('flearn', 'models', 'mnist', params['model'])
else:
  model_path = '%s.%s.%s.%s' % ('flearn', 'models', params['dataset'], params['model'])
mod = importlib.import_module(model_path)
learner = getattr(mod, 'Model')

# Set seeds
random.seed(1 + params['seed'])
np.random.seed(12 + params['seed'])
tf.set_random_seed(123 + params['seed'])

# Set server
if params['optimizer'] == 'fedavg':
  server = fedavg.Server(params, learner, dataset)  
if params['optimizer'] == 'fedprox':
  server = fedprox.Server(params, learner, dataset)
if params['optimizer'] == 'grouprox':
  server = grouprox.Server(params, learner, dataset)

Using Group prox to Train
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`
772 Clients in Total
243861 Parameters in Total
Instructions for updating:
Prefer Variable.assign which has equivalent behavior in 2.X.


Pre-training takes 26.722428798675537s seconds
SVD takes 0.45952510833740234s seconds
Delta w shape: (100, 243861) Matrix V shape: (243861, 3)
Ternary Cossim Matrix calculation takes 0.09747862815856934s seconds
Clustering takes 0.03478217124938965s seconds
Clustering Results: Counter({1: 45, 2: 23, 0: 16, 4: 9, 3: 7})
Clustering Inertia: 2.678375244140625


In [4]:
server.train()

Training with 20 workers ---


Group 0, clients ['schaeferj89', 'weeps']
Group 1, clients ['AllTheSausages', 'TinaS71', 'coriluvnthedon', 'Mrs_NickJ07', 'mneylon', 'davidj505', 'MacSheikh']
Group 2, clients ['m0po', 'debbier93', 'maddisondesigns', 'weelissa', 'AvonteNikole', 'janabelle_xo', 'nyc_specialist', 'WooopJess']
Group 3, clients ['steph_davies', 'marianaguidil']
Group 4, clients ['Japh']


Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

Group 0
At round 0 accuracy: 0.4933510638297872
At round 0 training accuracy: 0.5078288100208769
At round 0 training loss: 0.7110484339602722
Group 1
At round 0 accuracy: 0.8593700286350621
At round 0 training accuracy: 0.880852492028864
At round 0 training loss: 0.3744645367005634
Group 2
At round 0 accuracy: 0.7293150684931506
At round 0 training accuracy: 0.770196191575303
At round 0 training loss: 0.5551724155179284
Group 3
At round 0 accuracy: 0.4524793388429752
At round 0 training accuracy: 0.5176211453744494
At round 0 training loss: 0.6900799250156344
Group 4
At round 0 accuracy: 0.6832460732984293
At round 0 training accuracy: 0.6905692438402719
At round 0 training loss: 0.6492799088118334
At round 0 mean test accuracy: 0.7115747289014616 mean train accuracy: 0.73562648998421
The groups difference are: [1.91553522 2.13058498 2.14322334 1.80625589 1.79545015]
Begin group  0 training


Epoch:  10%|█         | 2/20 [00:00<00:01, 11.19it/s] 

...

Begin group  2 training


Epoch:  10%|█         | 2/20 [00:00<00:01, 11.77it/s] 

Begin group  3 training


Epoch:  10%|█         | 2/20 [00:00<00:01, 11.25it/s] 

Begin group  4 training


In [5]:
from collections import Counter

group_list = server.group_list
clients = server.clients
g = group_list[-1]
c_list = []
for c in clients:
  if c.group == g:
    c_list.append(c)

cnt = Counter()
for c in c_list:
  cnt += Counter(c.eval_data['y'])
print(cnt)
print(sum(cnt.values()))

Counter({'4': 1044, '0': 484})
1528


In [6]:
nd1 = np.arange(10).reshape(5,2)
nd2 = np.arange(10).reshape(5,2)
nd3 = np.array([3.]*10).reshape(5,2)
print((nd2+1.)/2.)
#print(nd2)

print(np.linalg.norm(nd2, axis=1, keepdims=True).shape)

[[0.5 1. ]
 [1.5 2. ]
 [2.5 3. ]
 [3.5 4. ]
 [4.5 5. ]]
(5, 1)


In [7]:
fname = 'sampling_interval.csv'
interval = 5
rounds = 200
with open(fname, 'w') as csv_f:
  csv_f.write(str(1)+'\n')
  for i in range(2, rounds+1):
    if i % interval == 0:
      csv_f.write(str(i))
    elif i == rounds:
      csv_f.write('200')
    csv_f.write('\n')